In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, random_split
import glob

from datasets import Poly, get_wine_data
from models import ModelSmall, ModelMicro
from ensemble import predict, evaluate, evaluate_ensemble_components

# Ensemble Member Summary

In [ ]:
experiment_key = "8316424c2c384b3eb96243f21d5778ae"
checkpoints = glob.glob(f"../checkpoints/{experiment_key}_fge*", recursive=False)

print(f"Found {len(checkpoints)} FGE checkpoints")
for checkpoint in checkpoints:
    print(checkpoint)

# Load Dataset

In [ ]:
train_ds, test_ds = get_wine_data()
train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)

# Load the Models

In [ ]:
device = torch.device("cuda:1")
models = []

for checkpoint in checkpoints:
    model = ModelSmall(input_dim=11, hidden_dim=32)
    model.load_state_dict(torch.load(checkpoint, map_location="cpu"))
    models.append(model.to(device))

# Performance of each model in ensemble

In [ ]:
df_post = evaluate_ensemble_components(train_loader, test_loader, device, models)

print("All Models")
print(df_post)

print("\ntrain_loss description:")
print(df_post["train_loss"].describe())

print("\nval_loss description:")
print(df_post["test_loss"].describe())

# Ensemble Performance

In [ ]:
print("Train performance: MSE_mean, MSE_median, average SD")
print(evaluate(train_loader, device, models))

print("Test performance: MSE_mean, MSE_median, average SD")
print(evaluate(test_loader, device, models))

In [ ]:
def ensemble_performance_table(loader):
    data = {"Size": [], "MSE with Mean": [], "MSE with Median": [], "SD": []}

    for ensemble in [models, models[:5], models[:3]]:
        a, b, c = evaluate(loader, device, ensemble)
        data["Size"].append(len(ensemble))
        data["MSE with Mean"].append(a)
        data["MSE with Median"].append(b)
        data["SD"].append(c)

    return pd.DataFrame(data)


print(ensemble_performance_table(train_loader))
print(ensemble_performance_table(test_loader))